In [10]:
# Pipeline (Download all the youtube videos given in the dataset in a folder)

def download_yt_vid(id, output_folder):
    
    import os
    from pytube import YouTube
    output_path = os.path.join(output_folder, id)
    
    if os.path.exists(output_path):
        return
    
    else:
        
        url_prefix = 'https://www.youtube.com/watch?v='
        url = url_prefix + id
        YouTube(url).streams.get_highest_resolution().download(output_path=output_folder, filename=id+'.mp4')



def download_yt_vids(anno_file_path, output_folder):
    
    import pickle as pkl
    
    f = open(anno_file_path, 'rb')
    anno = pkl.load(f)
    f.close()
    
    yt_ids = [ item['metadata']['yt_id'] for item in anno]
    yt_ids = list(set(yt_ids))
    not_done = []
    from tqdm import tqdm as tqdm
    for id in tqdm(yt_ids):
        try:
            download_yt_vid(id, output_folder)
        except:
            print("Error for " + id)
            not_done.append(id)
    return not_done

anno_path = '/workspace/work/O2ONet/data/annotation.pkl'
youtube_saving_path = '/workspace/data/data_folder/o2o/youtube2'
failed = download_yt_vids(anno_path, youtube_saving_path)

def prune_anno(anno_path, not_done):
    
    import pickle as pkl
    
    f = open(anno_path, 'rb')
    anno = pkl.load(f)
    f.close()
    
    pruned_anno = []
    
    for item in anno:
        yt_id = item['metadata']['yt_id']
        if yt_id not in not_done:
            pruned_anno.append(item)
    
    return pruned_anno
pruned_anno = prune_anno(anno_path, failed)
f = open('annotations_minus_unavailable_youtube_videos.pkl','wb')
import pickle as pkl
pkl.dump(pruned_anno, f)
f.close()

  3%|▎         | 12/434 [00:59<20:48,  2.96s/it] 

Error for kJxnwF9q41E


 11%|█▏        | 49/434 [03:47<12:48,  2.00s/it]  

Error for tAx_8TvArW4


 21%|██▏       | 93/434 [06:34<14:05,  2.48s/it]

...

 81%|████████  | 352/434 [25:30<04:34,  3.34s/it]  

Error for vR4jioUMlX8


 83%|████████▎ | 361/434 [26:03<02:48,  2.31s/it]

Error for TIlTe6TI4Cs


100%|██████████| 434/434 [31:03<00:00,  4.29s/it]


In [13]:
def extract_gif(frame_index, yt_id, youtube_folder, output_folder, window_size):
    
    import cv2
    import os
    
    video_loc = os.path.join(youtube_folder, yt_id + '.mp4')
    vid = cv2.VideoCapture(video_loc)
    
    frame_count = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))

    initial = frame_index - window_size
    final = frame_index + window_size

    if initial<0 or final>=frame_count:
        print("Issue for ", frame_index, " ",yt_id)
        return 0

    frames = []

    for i in range(frame_count):

        success, frame = vid.read()
        if i >= initial and i <= final and success:
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(rgb_frame)

    filename = yt_id + '_' + str(frame_index) + '_' + str(window_size) + '.gif'
    saving_location = os.path.join(output_folder, filename)

    import imageio
    fps = 4
    imageio.mimsave( saving_location, frames, fps=4)
    del vid
    return 1

def extract_gifs(anno_path, youtube_folder, output_folder, window_size):
    '''
    anno_path: Extracts all the annotations 
    '''

    import pickle as pkl
    
    f = open(anno_path, 'rb')
    anno = pkl.load(f)
    f.close()
    
    from tqdm import tqdm as tqdm
    failed = []

    for a in tqdm(anno):
        frame_index = int(a['metadata']['frame no.'])
        yt_id = a['metadata']['yt_id']
        ret = extract_gif(frame_index, yt_id, youtube_folder, output_folder, window_size)
        
        if not ret:
            failed.append(a)

    return failed

anno_path = '/workspace/work/O2ONet/data/annotations_minus_unavailable_yt_vids.pkl'
youtube_path = '/workspace/data/data_folder/o2o/youtube2'
output_folder = '/workspace/data/data_folder/o2o/gifs_11'
window_size = 5

failed = extract_gifs(anno_path, youtube_path, output_folder, window_size)

  0%|          | 0/2056 [00:00<?, ?it/s]

1567 1577 5 4107


  0%|          | 1/2056 [00:05<3:22:45,  5.92s/it]

2352 2362 5 4107


  0%|          | 2/2056 [00:11<3:15:18,  5.71s/it]

2353 2363 5 4107


  0%|          | 3/2056 [00:16<3:11:03,  5.58s/it]

2421 2431 5 4107


  0%|          | 3/2056 [00:21<4:03:04,  7.10s/it]


KeyboardInterrupt: 